# Scenario 2 (High Density) [5 Repetitions]

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

## Config

In [2]:
inputFile = '2kr-scalars-5rep.csv'
repetitionsCount = -1 # -1 = auto-detect
factors = ['R', 'T', 'm', 'D']

# Filter scalars
scalarsFilter = ['Floorplan.userCount', 'Floorplan.coveredUsers:sum', 'Floorplan.collisions:sum', 'Floorplan.activityTime:last']
# Performance indexes
perfIndexes = [
    ('coveredUsersPercent', 'percentage of covered users'),
    ('Floorplan.collisions:sum', 'total number of collisions'),
    ('Floorplan.activityTime:last', 'total broadcast time'),
]

## Load scalars

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

# Computed
factorsCount = len(factors)
configsCount = 2**factorsCount
totalSims = configsCount*repetitionsCount


display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', totalSims)
pd.set_option('display.max_columns', 2*configsCount*repetitionsCount+1)

Repetitions: 5


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values([*factors, 'repetition'], inplace=True)
count = 0
for index in scalars_wide.index:
    config = count // repetitionsCount
    scalars_wide.loc[index, 'config'] = config
    count += 1
scalars_wide = scalars_wide[['config', 'repetition', *factors, *scalarsFilter]]

In [6]:
# coverage
scalars_wide['coveredUsersPercent'] = scalars_wide['Floorplan.coveredUsers:sum'] / (scalars_wide['Floorplan.userCount'] - 1)

## 2kr

In [7]:
combinedFactors = list(chain(*[list(map(''.join, combinations(''.join(factors), i))) for i in range(1, factorsCount+1)]))
cols = ['exp', 'I'] + combinedFactors
data = [[e, 1] + list(chain(*[list(map(np.prod, combinations(r, i))) for i in range(1, len(r)+1)])) for r,e in zip([list(x) for x in product([-1, 1], repeat=factorsCount)], range(0, configsCount))]
factorsMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
for perfIndex, perfIndexDesc in perfIndexes:
    #print("2^k*r analysis for the", perfIndexDesc)
    display(HTML('<h3>2^k*r analysis for the ' + perfIndexDesc + '</h3>'))
    cols = ['exp'] + ['y' + str(i) for i in range(0, repetitionsCount)]
    data = [[i] + scalars_wide[scalars_wide.config == i][perfIndex].values.tolist() for i in range(0, configsCount)]
    ysMatrix = pd.DataFrame.from_records(data, columns=cols, index='exp')
    ysMatrix['mean'] = ysMatrix.mean(axis=1)
    for r in range(0, repetitionsCount):
        ysMatrix['e' + str(r)] = ysMatrix['y' + str(r)] - ysMatrix['mean']

    matrix = pd.concat([factorsMatrix, ysMatrix], axis=1)

    computeCols = ['idx', 'I'] + combinedFactors + ['e' + str(r) for r in range(0, repetitionsCount)]
    data = [['Sum'], ['Mean (Qi)'], ['SSQ']]
    SSE = 0
    for col in computeCols[1:]:
        sumvalue = np.sum(matrix[col].values * matrix['mean'].values)
        data[0].append(sumvalue)
        data[1].append(sumvalue / configsCount)
        if col in ['e' + str(r) for r in range(0, repetitionsCount)]:
            ssq = np.sum(matrix[col].values * matrix[col].values)
            data[2].append(ssq)
            SSE += ssq
        else:
            data[2].append('')
    computeMatrix = pd.DataFrame.from_records(data, columns=computeCols, index='idx')

    data = [['(2^k)*r*(qi)^2'], ['Variation'], ['Variation%']]
    SST = SSE
    for col in combinedFactors:
        qi = computeMatrix.loc['Mean (Qi)', col]
        r2kqi2 = configsCount*repetitionsCount*(qi**2)
        data[0].append(r2kqi2)
        SST += r2kqi2
    data[0].append(SSE)
    idx = 1
    for col in combinedFactors:
        r2kqi2 = data[0][idx]
        var = r2kqi2 / SST
        data[1].append(var)
        data[2].append('{:.2%}'.format(var))
        idx += 1
    data[1].append(SST)
    data[2].append('{:.2%}'.format(SSE/SST))
    resultMatrix = pd.DataFrame.from_records(data, columns=['idx'] + combinedFactors + ['SSE/SST'], index='idx')

    display(matrix)
    display(computeMatrix)
    display(resultMatrix)

    print('SSE =', SSE)
    print('SST =', SST)
    print('Unexplained Variation =', '{:.2%}'.format(SSE/SST))
    print('####################################')
    print()

,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,mean,e0,e1,e2,e3,e4
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,0.891784,0.889780,0.875752,0.897796,0.813627,0.873747,0.018036,0.016032,0.002004,0.024048,-0.060120
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,0.987976,0.975952,0.907816,0.985972,0.979960,0.967535,0.020441,0.008417,-0.059719,0.018437,0.012425
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,0.969940,0.973948,0.967936,0.967936,0.961924,0.968337,0.001603,0.005611,-0.000401,-0.000401,-0.006413
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,0.995992,1.000000,0.991984,0.997996,0.993988,0.995992,0.000000,0.004008,-0.004008,0.002004,-0.002004
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,0.929860,0.855711,0.843687,0.903808,0.879760,0.882565,0.047295,-0.026854,-0.038878,0.021242,-0.002806
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,0.959920,0.971944,0.997996,0.957916,0.416834,0.860922,0.098998,0.111022,0.137074,0.096994,-0.444088
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,0.989980,0.991984,0.993988,0.987976,0.993988,0.991583,-0.001603,0.000401,0.002405,-0.003607,0.002405
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,0.833667,0.745491,0.474950,0.923848,0.779559,0.751503,0.082164,-0.006012,-0.276553,0.172345,0.028056


,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4
idx,,,,,,,,,,,,,,,,,,,,,
Sum,14.332665,-0.748697,0.179559,0.561122,0.524248,0.320641,-0.181162,0.307816,0.336673,0.041683,-0.296593,0.086573,0.311022,-0.224449,0.147495,-0.04489,0.213532,0.153941,-0.229290,0.286099,-0.424282
Mean (Qi),0.895792,-0.046794,0.011222,0.03507,0.032766,0.02004,-0.011323,0.019238,0.021042,0.002605,-0.018537,0.005411,0.019439,-0.014028,0.009218,-0.002806,0.013346,0.009621,-0.014331,0.017881,-0.026518
SSQ,,,,,,,,,,,,,,,,,0.043727,0.018900,0.146172,0.068157,0.212933


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,0.175171,0.010075,0.098393,0.085886,0.032128,0.010256,0.02961,0.035422,0.000543,0.02749,0.002342,0.03023,0.015743,0.006798,0.00063,0.489889
Variation,0.166733,0.00959,0.093654,0.081749,0.030581,0.009762,0.028183,0.033715,0.000517,0.026166,0.002229,0.028773,0.014985,0.006471,0.000599,1.050606
Variation%,16.67%,0.96%,9.37%,8.17%,3.06%,0.98%,2.82%,3.37%,0.05%,2.62%,0.22%,2.88%,1.50%,0.65%,0.06%,46.63%


SSE = 0.4898887956273269
SST = 1.0506062224649697
Unexplained Variation = 46.63%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,mean,e0,e1,e2,e3,e4
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,323.0,351.0,314.0,356.0,293.0,327.4,-4.4,23.6,-13.4,28.6,-34.4
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,225.0,197.0,223.0,222.0,247.0,222.8,2.2,-25.8,0.2,-0.8,24.2
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,680.0,676.0,665.0,658.0,629.0,661.6,18.4,14.4,3.4,-3.6,-32.6
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,341.0,318.0,362.0,335.0,315.0,334.2,6.8,-16.2,27.8,0.8,-19.2
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,313.0,274.0,294.0,285.0,352.0,303.6,9.4,-29.6,-9.6,-18.6,48.4
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,178.0,146.0,130.0,171.0,70.0,139.0,39.0,7.0,-9.0,32.0,-69.0
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,697.0,618.0,577.0,655.0,638.0,637.0,60.0,-19.0,-60.0,18.0,1.0
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,407.0,406.0,445.0,416.0,420.0,418.8,-11.8,-12.8,26.2,-2.8,1.2
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,585.0,795.0,861.0,558.0,616.0,683.0,-98.0,112.0,178.0,-125.0,-67.0


...

SSE = 1138084.0
SST = 32685571.987499997
Unexplained Variation = 3.48%
####################################



,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,y0,y1,y2,y3,y4,mean,e0,e1,e2,e3,e4
exp,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,38.0,35.0,45.0,25.0,34.0,35.4,2.6,-0.4,9.6,-10.4,-1.4
1,1,-1,-1,-1,1,1,1,-1,1,-1,-1,-1,1,1,1,-1,60.0,54.0,65.0,60.0,55.0,58.8,1.2,-4.8,6.2,1.2,-3.8
2,1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,1,-1,34.0,29.0,38.0,38.0,28.0,33.4,0.6,-4.4,4.6,4.6,-5.4
3,1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,46.0,50.0,40.0,56.0,45.0,47.4,-1.4,2.6,-7.4,8.6,-2.4
4,1,-1,1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,-1,133.0,85.0,119.0,99.0,109.0,109.0,24.0,-24.0,10.0,-10.0,0.0
5,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,1,-1,1,232.0,112.0,149.0,116.0,127.0,147.2,84.8,-35.2,1.8,-31.2,-20.2
6,1,-1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,1,-1,1,96.0,78.0,96.0,103.0,111.0,96.8,-0.8,-18.8,-0.8,6.2,14.2
7,1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,1,-1,102.0,113.0,97.0,92.0,111.0,103.0,-1.0,10.0,-6.0,-11.0,8.0
8,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,-1,-1,8.0,14.0,8.0,14.0,12.0,11.2,-3.2,2.8,-3.2,2.8,0.8


,I,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,e0,e1,e2,e3,e4
idx,,,,,,,,,,,,,,,,,,,,,
Sum,817.8,-444.2,360.6,-68.6,89.8,-201.4,71.0,-73.8,-45.8,5.4,-56.6,40.2,-8.6,26.2,-33.8,11.4,15251.36,-9302.640,734.56,-5244.8400,-1438.4400
Mean (Qi),51.1125,-27.7625,22.5375,-4.2875,5.6125,-12.5875,4.4375,-4.6125,-2.8625,0.3375,-3.5375,2.5125,-0.5375,1.6375,-2.1125,0.7125,953.21,-581.415,45.91,-327.8025,-89.9025
SSQ,,,,,,,,,,,,,,,,,8209.44,2433.440,448.04,2326.8400,1025.0400


,R,T,m,D,RT,Rm,RD,Tm,TD,mD,RTm,RTD,RmD,TmD,RTmD,SSE/SST
idx,,,,,,,,,,,,,,,,
(2^k)*r*(qi)^2,61660.5125,40635.1125,1470.6125,2520.0125,12675.6125,1575.3125,1702.0125,655.5125,9.1125,1001.1125,505.0125,23.1125,214.5125,357.0125,40.6125,14442.8
Variation,0.442049,0.291316,0.010543,0.018066,0.090872,0.011294,0.012202,0.004699,0.000065,0.007177,0.00362,0.000166,0.001538,0.002559,0.000291,139487.9875
Variation%,44.20%,29.13%,1.05%,1.81%,9.09%,1.13%,1.22%,0.47%,0.01%,0.72%,0.36%,0.02%,0.15%,0.26%,0.03%,10.35%


SSE = 14442.800000000001
SST = 139487.9875
Unexplained Variation = 10.35%
####################################

